In [1]:
import random 
import numpy as np 
import pandas as pd  

In [2]:
def set_seed(seed): 
    random.seed(seed) 
    np.random.seed(seed) 
    
set_seed(42) 

In [3]:
book_descriptions = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/book_descriptions.csv')
book_genres = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/book_genres.csv') 
books = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/books.csv') 
genres = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/genres.csv') 
sample_submission = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/sample_submission.csv') 
test = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/test.csv') 
train = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/train.csv') 
users = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/individual_data/users.csv') 

# Подход № 1

In [4]:
train = train[train['has_read'] == 1].copy() 
global_mean = train['rating'].mean() # глобальное среднее. Если не будет информации то будем возвращать в качетсве предикта его.

In [5]:
train = train.merge(books[["book_id", "author_id"]], on="book_id", how="left")
test  = test.merge(books[["book_id", "author_id"]], on="book_id", how="left")

book_genres_full = genres.copy()

In [6]:
book_genres_full

,genre_id,genre_name,books_count
0,1,Бизнес-книги,254
1,2,Банковское-дело,62
2,3,Бухучет-налогообложение-аудит,9
3,5,Делопроизводство,17
4,6,Зарубежная-деловая-литература,6
...,...,...,...
436,1375,Графические-романы,1
437,1381,Маньхуа,1
438,1384,Психологический-роман,6
439,1388,Славянское-фэнтези,4


In [10]:
# вычисляем средний рейтинг, std рейтинга, количество голосований пользователя

user_stats_full = (
    train.groupby("user_id")["rating"]
        .agg(["mean", "std", "count"])
        .reset_index()
        .rename(columns={"mean": "user_mean", "std": "user_std", "count": "user_count"})
)

user_stats_full["user_std"] = user_stats_full["user_std"].fillna(0.0)

In [12]:
book_genres_full = book_genres_full.merge(book_genres, on='genre_id', how='left')
train_expanded = train.merge(book_genres_full, on="book_id", how="left")

In [15]:
# вычисляем средний рейтинг и количество оценок для жанра с которым имел дело конкретный пользователь.
user_genre_stats = (
    train_expanded.groupby(["user_id", "genre_id"])["rating"]
        .agg(["mean", "count"])
        .reset_index()
        .rename(columns={"mean": "user_genre_mean", "count": "user_genre_count"})
)

In [16]:
user_genre_stats

,user_id,genre_id,user_genre_mean,user_genre_count
0,150,13,8.000000,1
1,150,90,8.000000,2
2,150,405,8.000000,1
3,150,412,10.000000,1
4,150,424,10.000000,1
...,...,...,...,...
144681,10987250,1224,8.000000,1
144682,10987250,1280,8.000000,1
144683,10987250,1316,10.000000,1
144684,10987250,1326,8.000000,1


In [17]:
# вычисляем средний рейтинг оценки для автора с которым имел дело конкретный пользователю.
user_author_mean = (
    train.groupby(["user_id", "author_id"])["rating"]
        .mean()
        .reset_index()
        .rename(columns={"rating": "user_author_mean"})
)

In [22]:
user_author_mean

,user_id,author_id,user_author_mean
0,150,18629,10.0
1,150,49184,8.0
2,150,222410,8.0
3,281,15885,6.0
4,281,27112,4.0
...,...,...,...
110073,10987250,303504,10.0
110074,10987250,419890,8.0
110075,10987250,570002,8.0
110076,10987250,626080,8.0


In [19]:
user_mean = user_stats_full[["user_id", "user_mean"]].copy()

In [23]:
# средняя оценка книги в целом. 
book_mean = (
    train.groupby("book_id")["rating"]
        .mean()
        .reset_index()
        .rename(columns={"rating": "book_mean"})
)

In [30]:
book_mean

,book_id,book_mean
0,20,6.5
1,35,4.0
2,53,8.0
3,54,8.0
4,69,6.0
...,...,...
34535,8493935,9.5
34536,8508008,8.0
34537,8508119,8.0
34538,8536319,10.0


In [26]:
# средняя оценка автора в целом (не по пользователю)
author_global_mean = (
    train.groupby("author_id")["rating"]
        .mean()
        .reset_index()
        .rename(columns={"rating": "author_global_mean"})
)

In [27]:
# средняя оценка жанра в целом.
genre_global_mean = (
    train_expanded.groupby("genre_id")["rating"]
        .mean()
        .reset_index()
        .rename(columns={"rating": "genre_global_mean"})
)

In [31]:
genre_global_mean

,genre_id,genre_global_mean
0,1,7.466667
1,2,7.900000
2,3,4.333333
3,5,7.363636
4,6,7.500000
...,...,...
428,1375,7.750000
429,1381,8.000000
430,1384,6.537037
431,1388,8.112676


In [28]:
# среднее средних оценок жанра для каждой уникальной книги.
book_genre_global_mean = (
    book_genres_full
        .merge(genre_global_mean, on="genre_id", how="left")
        .groupby("book_id")["genre_global_mean"]
        .mean()
        .reset_index()
        .rename(columns={"genre_global_mean": "book_genre_global_mean"})
)

In [33]:
book_genre_global_mean

,book_id,book_genre_global_mean
0,20,7.577379
1,35,7.720203
2,53,7.437326
3,54,7.656281
4,69,7.622958
...,...,...
50484,8508008,7.588869
50485,8508107,7.958169
50486,8508119,7.529083
50487,8536319,7.357652


In [34]:
test_multi = test.merge(book_genres_full, on="book_id", how="left")

test_multi = test_multi.merge(
    user_genre_stats,
    on=["user_id", "genre_id"],
    how="left"
)

In [35]:
test_multi

,user_id,book_id,author_id,genre_id,genre_name,books_count,user_genre_mean,user_genre_count
0,281,2461928,849103,1242,Героическое-фэнтези,554,8.000000,2.0
1,1250,31957,1900171,447,Советская-литература,347,6.000000,3.0
2,1250,31957,1900171,1133,Приключения,579,8.000000,1.0
3,1250,31957,1900171,1314,Отечественная-фантастика,1452,3.500000,2.0
4,4241,196603,166428,433,Современная-зарубежная-литература,5937,7.411765,17.0
...,...,...,...,...,...,...,...,...
6415,6459040,443150,202274,433,Современная-зарубежная-литература,5937,NaN,NaN
6416,6459040,443150,202274,1224,Мистика,576,NaN,NaN
6417,10335850,2435229,16611,1222,Современная-русская-литература,1745,8.000000,5.0
6418,10987250,1753284,429832,1217,Современная-проза,2429,8.800000,5.0


In [36]:
test_multi["user_genre_mean"]  = test_multi["user_genre_mean"].fillna(global_mean)
test_multi["user_genre_count"] = test_multi["user_genre_count"].fillna(0)

Расчитаем вес на основе жанров прочитанных пользоваетелем: 

1. Добавим колонку `user_genre_count_smooth` добавив к `user_genre_count` еденицу, чтобы при расчете веса избежать деления на 0.
2. Расчитаем общее количество прочтений жанров пользователем.
3. Расчитаем вес как количество прочтений конкретного жанра пользователем деленное на общее количество прочитанных жанров.
4. Нормализуем вес, поделив на сумму весов для одинкавой пары `user_id : book_id`. 


In [37]:
test_multi["user_genre_count_smooth"] = test_multi["user_genre_count"] + 1

test_multi["user_total_genre_reads"] = (
    test_multi.groupby("user_id")["user_genre_count_smooth"].transform("sum")
)

test_multi["genre_weight"] = (
    test_multi["user_genre_count_smooth"] / test_multi["user_total_genre_reads"]
)

test_multi["normalized_weight"] = (
    test_multi["genre_weight"] /
    test_multi.groupby(["user_id","book_id"])["genre_weight"].transform("sum")
).fillna(0)

In [38]:
test_multi 

,user_id,book_id,author_id,genre_id,genre_name,books_count,user_genre_mean,user_genre_count,user_genre_count_smooth,user_total_genre_reads,genre_weight,normalized_weight
0,281,2461928,849103,1242,Героическое-фэнтези,554,8.000000,2.0,3.0,3.0,1.000000,1.000000
1,1250,31957,1900171,447,Советская-литература,347,6.000000,3.0,4.0,9.0,0.444444,0.444444
2,1250,31957,1900171,1133,Приключения,579,8.000000,1.0,2.0,9.0,0.222222,0.222222
3,1250,31957,1900171,1314,Отечественная-фантастика,1452,3.500000,2.0,3.0,9.0,0.333333,0.333333
4,4241,196603,166428,433,Современная-зарубежная-литература,5937,7.411765,17.0,18.0,24.0,0.750000,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...
6415,6459040,443150,202274,433,Современная-зарубежная-литература,5937,7.663201,0.0,1.0,3.0,0.333333,0.333333
6416,6459040,443150,202274,1224,Мистика,576,7.663201,0.0,1.0,3.0,0.333333,0.333333
6417,10335850,2435229,16611,1222,Современная-русская-литература,1745,8.000000,5.0,6.0,6.0,1.000000,1.000000
6418,10987250,1753284,429832,1217,Современная-проза,2429,8.800000,5.0,6.0,11.0,0.545455,0.545455


Расчитаем взвешенный вес - домножим расчитанный вес на среденюю оценку жанра конкретным пользователем (или же средняя оценка жанра по мненюю данного пользователя) 

In [39]:
test_multi["weighted_genre_mean"] = (
    test_multi["normalized_weight"] * test_multi["user_genre_mean"]
)

user_multi_genre_mean = (
    test_multi.groupby(["user_id", "book_id"])["weighted_genre_mean"]
        .sum()
        .reset_index()
        .rename(columns={"weighted_genre_mean": "genre_weighted_prediction"})
)

In [40]:
user_multi_genre_mean

,user_id,book_id,genre_weighted_prediction
0,281,2461928,8.000000
1,1250,31957,5.611111
2,4241,196603,7.336457
3,5140,468894,7.663201
4,7781,2141951,7.663201
...,...,...,...
2889,5689120,2215170,7.663201
2890,5699230,4627999,8.200000
2891,6459040,443150,7.663201
2892,10335850,2435229,8.000000


Соединим полученные статистики в один фрейм 

In [41]:
test2 = test.merge(user_multi_genre_mean, on=["user_id", "book_id"], how="left")
test2["genre_weighted_prediction"] = test2["genre_weighted_prediction"].fillna(global_mean)

test2 = test2.merge(user_author_mean, on=["user_id", "author_id"], how="left")

test2 = test2.merge(user_mean, on="user_id", how="left")
test2 = test2.merge(user_stats_full[["user_id", "user_std"]], on="user_id", how="left")
test2["user_std"] = test2["user_std"].fillna(0.0)

test2 = test2.merge(book_mean, on="book_id", how="left")

test2 = test2.merge(author_global_mean, on="author_id", how="left")
test2["author_global_mean"] = test2["author_global_mean"].fillna(global_mean)

test2 = test2.merge(book_genre_global_mean, on="book_id", how="left")
test2["book_genre_global_mean"] = test2["book_genre_global_mean"].fillna(global_mean)

Расчитаем маркеры: 

1. `khown_user` - `True`  если пользователь встречался в `train` выборке (мы видели его ранее), инчае `False`.
2. `known_book` - `True` если книга встречалась в `train` выборке ранее, иначе `False`.

In [42]:
known_users = set(train["user_id"].unique())
known_books = set(train["book_id"].unique())

test2["known_user"] = test2["user_id"].isin(known_users)
test2["known_book"] = test2["book_id"].isin(known_books)

In [69]:
test2

,user_id,book_id,author_id,genre_weighted_prediction,user_author_mean,user_mean,user_std,book_mean,author_global_mean,book_genre_global_mean,known_user,known_book,rating_predict
0,281,2461928,849103,8.000000,NaN,7.714286,1.454058,10.000000,6.666667,8.165793,True,True,8.420751
1,1250,31957,1900171,5.611111,NaN,7.375000,3.348086,NaN,6.000000,7.829067,True,False,6.500812
2,4241,196603,166428,7.336457,NaN,8.000000,1.971222,8.388889,8.330097,7.704309,True,True,8.063240
3,5140,468894,231907,7.663201,NaN,9.500000,1.000000,NaN,10.000000,7.651603,True,False,8.590805
4,7781,2141951,180063,7.663201,NaN,NaN,0.000000,7.190476,7.292683,7.553533,False,True,7.549957
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2889,5689120,2215170,648837,7.663201,NaN,10.000000,0.000000,7.487805,7.853448,7.681294,True,True,8.337342
2890,5699230,4627999,1078900,8.200000,NaN,7.263158,1.790162,NaN,8.689655,7.796678,True,False,8.264867
2891,6459040,443150,202274,7.663201,NaN,9.285714,0.994490,6.833333,8.084337,7.607441,True,True,8.093219
2892,10335850,2435229,16611,8.000000,NaN,6.969697,2.555446,NaN,8.500000,7.580951,True,False,8.038875


`weighted_avg` - функция которая позволяет расчитывать итоговый рейтинг на основе набора информации.
Она примнимает на вход данные вида: 

| Источник  | Значение | Вес |
| --------- | -------- | --- |
| user_mean | 4.6      | 0.4 | 
| genre     | 4.2      | 0.3 |
| book      | 4.8      | 0.3 |

или же: [
  (value_1, weight_1),
  (value_2, weight_2),
  ...
]

`weighted_avg` - убирает NAN, если после читки не остается значений возвращаем global mean. Нормализует веса на их сумму. Считает взвешенное средние, что и является итоговым рейтингом.


`weighted_avg` = $w_1\cdot x_1 + w_2\cdot y_2 + ... + w_6\cdot y_6$

In [44]:
def weighted_avg(candidates):
    vals = [(v, w) for (v, w) in candidates if pd.notna(v)]
    if not vals:
        return global_mean
    ws = np.array([w for _, w in vals], dtype=float)
    ws = ws / ws.sum()
    xs = np.array([v for v, _ in vals], dtype=float)
    return float((xs * ws).sum())

`rebalance_user_vs_global` - функция для балансировки user based весов и global based весов, она позволяет сравнить влияние признаков основанных на информации пользователя и признаков основанных на глобальной информации. Данная функция схожа с механизмом attention.

In [45]:
def rebalance_user_vs_global(cands, user_keys, global_keys, force_equal=True):
    """
    cands: list of tuples (key, value, weight)
    user_keys/global_keys: which keys belong to which group
    force_equal: если True -> сумма весов user == сумма весов global (только по доступным сигналам!)
    """
    # оставим только не-NaN
    cands = [(k, v, w) for (k, v, w) in cands if pd.notna(v)]
    if not cands:
        return []

    user = [(k, v, w) for (k, v, w) in cands if k in user_keys]
    glob = [(k, v, w) for (k, v, w) in cands if k in global_keys]
    other = [(k, v, w) for (k, v, w) in cands if (k not in user_keys and k not in global_keys)]

    # если одной группы вообще нет — нечего уравнивать
    if not user or not glob or not force_equal:
        return [(v, w) for (_, v, w) in cands]

    sum_u = sum(w for _, _, w in user)
    sum_g = sum(w for _, _, w in glob)

    # хотим: sum_u_new == sum_g_new == (sum_u + sum_g)/2
    target = 0.5 * (sum_u + sum_g)

    # масштабирование внутри групп, сохраняя пропорции
    scale_u = target / sum_u if sum_u > 0 else 1.0
    scale_g = target / sum_g if sum_g > 0 else 1.0

    user_new = [(v, w * scale_u) for _, v, w in user]
    glob_new = [(v, w * scale_g) for _, v, w in glob]
    other_new = [(v, w) for _, v, w in other]

    return user_new + glob_new + other_new

`conditional_pred` - основная функция формирования рейтинга (роутер). 
1. Сначала мы извлекаем все признаки с которыми будем работать.
    Главные 3 признака - новая ли книга, новый ли пользователь, std оценки пользователя.
2. Если пользователь уже встречался в train и книга уже встречалась в train. то мы можем ипользовать все их статистики.
   Распределим веса эвристическим методом чтобы их сумма была равна 1.
   Если *user_std* > 4 (подбираемый гипер параметр через сабмит), то стоит сбалансировать веса (влияние) *user_based* и *global_based* информации.    Иначе просто используем признаки и веса без ребалансировки. Получившиеся веса и признаки закидываем в `weighted_avg` для получения финального рейтинга для строки.
4. Если пользователь известен, но книга - нет. То используем признаки которые основаны только на пользователе, но не на книге.
5. Если неизвестен пользователь и неизвестна книга, то используем глобальную статистику по автору и жанру не опираясь на *user_based* и *book_based* признаки. 

In [46]:
def conditional_pred(row):
    ku = row["known_user"]
    kb = row["known_book"]
    ustd = row["user_std"]

    # сигналы
    genre_pred    = row["genre_weighted_prediction"]      # число
    author_user   = row["user_author_mean"]               # NaN возможно
    user_m        = row["user_mean"]                      # NaN возможно
    book_m        = row["book_mean"]                      # NaN возможно
    author_global = row["author_global_mean"]             # число
    genre_global  = row["book_genre_global_mean"]         # число

    # какие сигналы "связанные с юзером"
    USER_KEYS = {"genre_pred", "author_user", "user_m"}
    GLOBAL_KEYS = {"author_global", "genre_global", "book_m"}

    # --- 1) known user & known book ---
    if ku and kb:
        base = [
            ("genre_pred",    genre_pred,    0.30),
            ("author_user",   author_user,   0.20),
            ("user_m",        user_m,        0.15),
            ("book_m",        book_m,        0.15),
            ("author_global", author_global, 0.10),
            ("genre_global",  genre_global,  0.10),
        ]

        # твоя идея: если user_std > 4, сделать user-часть == global-части
        if ustd > 4:
            cands = rebalance_user_vs_global(base, USER_KEYS, GLOBAL_KEYS, force_equal=True)
        else:
            cands = [(v, w) for (_, v, w) in base]

        return weighted_avg(cands)

    # --- 2) known user, new book ---
    if ku and not kb:
        base = [
            ("genre_pred",    genre_pred,    0.40),
            ("author_user",   author_user,   0.35),
            ("user_m",        user_m,        0.15),
            ("author_global", author_global, 0.10),
            ("genre_global",  genre_global,  0.10),
        ]

        if ustd > 4:
            cands = rebalance_user_vs_global(base, USER_KEYS, GLOBAL_KEYS, force_equal=True)
        else:
            cands = [(v, w) for (_, v, w) in base]

        return weighted_avg(cands)

    # --- 3) new user, known book ---
    if (not ku) and kb:
        base = [
            ("book_m",        book_m,        0.40),
            ("author_global", author_global, 0.30),
            ("genre_global",  genre_global,  0.30),
        ]
        cands = [(v, w) for (_, v, w) in base]
        return weighted_avg(cands)

    # --- 4) cold start ---
    base = [
        ("author_global", author_global, 0.50),
        ("genre_global",  genre_global,  0.50),
    ]
    cands = [(v, w) for (_, v, w) in base]
    return weighted_avg(cands)

#### Предсказание

In [47]:
test2["rating_predict"] = test2.apply(conditional_pred, axis=1)
test2["rating_predict"] = np.clip(test2["rating_predict"], 0, 10) * 1.03 # клипаем итоговый рейтинг

test2[["user_id","book_id","rating_predict"]].to_csv(
    "submission_superblend_conditional_3.0roundedto0.csv", index=False
)

print("DONE! submission_superblend_conditional_3.0roundedto0.csv saved")

DONE! submission_superblend_conditional_3.0roundedto0.csv saved


# Подход № 2

In [48]:
# Очистка
train = train[train['has_read'] == 1]

In [50]:
# Эти числа можно перебирать, чтобы выбить 0.766+
K_USER = 10   # Сила сглаживания для юзеров
K_ITEM = 5    # Сила сглаживания для книг
K_GENRE = 5   # Сила сглаживания для жанров

In [51]:
global_mean = train['rating'].mean()
print(f"Global Mean: {global_mean:.4f}")

Global Mean: 7.6633


Будем использовать формулу сглаживания для определения bias`a книги, пользователя: 
$$
Bias = \frac{\sum User Ratings + K \cdot GlobalMean}{Count + K} - GlobalMean
$$

$K$ - со скольки оценок мы считаем рейтинг пользователя весомым. \
$Count$ - Количество оценок пользователем. \
$\sum User Rating$ - Общая оценка.

In [52]:
# Формула сглаживания: (Sum(Ratings) + K * GlobalMean) / (Count + K) - GlobalMean
user_stats = train.groupby('user_id')['rating'].agg(['sum', 'count'])
user_stats['shrunk_bias'] = (user_stats['sum'] + K_USER * global_mean) / (user_stats['count'] + K_USER) - global_mean
user_bias_map = user_stats['shrunk_bias'].to_dict()

In [54]:
# Сначала считаем смещение книги БЕЗ учета юзера (rating - global - user_bias)
# Это важно! Мы хотим очистить оценку книги от настроения юзера.
train = train.merge(user_stats[['shrunk_bias']], on='user_id', how='left')
train['rating_resid'] = train['rating'] - global_mean - train['shrunk_bias']

item_stats = train.groupby('book_id')['rating_resid'].agg(['sum', 'count'])
item_stats['shrunk_bias'] = item_stats['sum'] / (item_stats['count'] + K_ITEM) # тут упрощенное сглаживание к 0
item_bias_map = item_stats['shrunk_bias'].to_dict()

Основная идея из которой складывается оценка для конкретного пользователя: 

rating = global mean + user bias + book bias + genre bias + noise.

**User bias** - особенность оценок конкретного пользователя: user_bias = average(user_ratings) - global_mean.
Однако если пользователь оценил слишком мало книг, то его bias нельзя считать хорошем.
Поэтому необходимо применять сглажиание вида: (user_sum + K * global_mean) / (count + K).
Возьмем K = 10:

- пока нет 10 оценок → считаем, что пользователь «похож на среднего»
- чем больше оценок → тем меньше влияние global_mean

**Book bias** - особенность конкретной книги на основе рейтинга, некоторый книги нраится больше, некоторые меньше. 

**Genre bias** - особенный. Genre bias = `вкус пользователя`, который можно расчитать вычитая из рейтинг все лишние факторы: 

`вкус пользователя к жанру`(остаток) = rating - global_mean - user_bias - book_bias 

Положительный остаток означает что жанр нравится, отрицательный - значить что не нравится.

In [55]:
# --- Вкусовщина ---
# Для каждой оценки в трейне узнаем жанры книги
train_genres = train[['user_id', 'book_id', 'rating_resid']].merge(book_genres, on='book_id')

# Считаем остаток после вычета Книги: Resid_2 = Rating - Global - UserBias - ItemBias
# Нам нужно понять: после того как мы учли крутость книги, почему юзер поставил выше/ниже? Из-за жанра!
train_genres['item_bias'] = train_genres['book_id'].map(item_bias_map).fillna(0)
train_genres['genre_resid'] = train_genres['rating_resid'] - train_genres['item_bias']

In [56]:
# Считаем смещение юзера по каждому жанру
ug_stats = train_genres.groupby(['user_id', 'genre_id'])['genre_resid'].agg(['sum', 'count'])
ug_stats['shrunk_bias'] = ug_stats['sum'] / (ug_stats['count'] + K_GENRE)
# Превращаем в словарь (user_id, genre_id) -> bias
ug_map = ug_stats['shrunk_bias'].to_dict()

In [58]:
# --- ПРЕДСКАЗАНИЕ ---

# Кэшируем жанры для быстрого доступа: book_id -> [genre_id, genre_id...]
book2genres = book_genres.groupby('book_id')['genre_id'].apply(list).to_dict()

def predict_smart(row):
    uid = row['user_id']
    bid = row['book_id']
    
    # 1. База + Юзер
    u_bias = user_bias_map.get(uid, 0.0) # Если юзера нет, 0
    pred = global_mean + u_bias
    
    # 2. Книга
    i_bias = item_bias_map.get(bid, 0.0)
    pred += i_bias
    
    # 3. Жанры (Среднее смещение по всем жанрам этой книги для этого юзера)
    genres = book2genres.get(bid, [])
    if genres:
        g_biases = []
        for g in genres:
            # Ищем пару (User, Genre)
            val = ug_map.get((uid, g))
            if val is not None:
                g_biases.append(val)
        
        if g_biases:
            # Добавляем среднее отклонение по жанрам
            pred += np.mean(g_biases)
            
    return np.clip(pred, 0, 10)

In [59]:
print("Генерация V2...")
test['rating_predict'] = test.apply(predict_smart, axis=1)

Генерация V2...


In [63]:
test.to_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/submission_algo2.csv')

In [64]:
# base_pred = pd.read_csv('/kaggle/input/eto-ai-nto-suchka/submission.csv') <- baseline submission

bias_pred = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/submission_algo2.csv')
my_pred = pd.read_csv('/Users/ilagulakin/Desktop/AI & code/NTO 2025/individual/submission_superblend_conditional_3.0roundedto0.csv')

In [66]:
super_mega_blend = pd.DataFrame({
    'user_id': bias_pred['user_id'],
    'book_id': bias_pred['book_id'],
    'rating_predict': my_pred['rating_predict']*0.70 + bias_pred['rating_predict']*0.3}) #+ base_pred['rating_predict']*0.125})

super_mega_blend.to_csv('SUPER_MEGA_BLEND_SIGMA_SOTA_KILLER_GONCHAROV_KIRPICHENKO_KHLOPOTNUKH_SOLUTION676767.csv', index=False)
super_mega_blend.head()

,user_id,book_id,rating_predict
0,281,2461928,7.307216
1,1250,31957,5.754325
2,4241,196603,7.102197
3,5140,468894,7.446479
4,7781,2141951,6.586453
